In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [2]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies, crosstab
import seaborn as sns # iris 데이터 가져오기
from sklearn import datasets # iris 데이터 (X,y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [23]:
# 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                    test_size=0.2,
                                                    stratify= iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [24]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 64)                320       
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 50)                6450      
                                                                 
 dense_19 (Dense)            (None, 30)                1530      
                                                                 
 dense_20 (Dense)            (None, 3)                 93        
                                                                 
Total params: 16,713
Trainable params: 16,713
Non-trainable params: 0
_________________________________________________________________


In [26]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,
               epochs=200,
               validation_split=0.1,
               callbacks=[earlystopping])

Epoch 1/200
4/4 [==============================] - 0s 62ms/step - loss: 0.0962 - accuracy: 0.9722 - val_loss: 0.0777 - val_accuracy: 0.9167
Epoch 2/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0947 - accuracy: 0.9352 - val_loss: 0.0606 - val_accuracy: 1.0000
Epoch 3/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0751 - accuracy: 0.9722 - val_loss: 0.0620 - val_accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0746 - accuracy: 0.9722 - val_loss: 0.0412 - val_accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0704 - accuracy: 0.9722 - val_loss: 0.0401 - val_accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0662 - accuracy: 0.9722 - val_loss: 0.0435 - val_accuracy: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0670 - accuracy: 0.9722 - val_loss: 0.0365 - val_accuracy: 1.0000
Epoch 8/200
4/4 [========

4/4 [==============================] - 0s 9ms/step - loss: 0.0570 - accuracy: 0.9815 - val_loss: 0.0347 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0739 - accuracy: 0.9537 - val_loss: 0.0355 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0724 - accuracy: 0.9815 - val_loss: 0.0244 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0601 - accuracy: 0.9907 - val_loss: 0.0242 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0594 - accuracy: 0.9815 - val_loss: 0.0210 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0641 - accuracy: 0.9815 - val_loss: 0.0224 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0658 - accuracy: 0.9722 - val_loss: 0.0233 - val_accuracy: 1.0000
Epoch 66/200
4/4 [==============

4/4 [==============================] - 0s 13ms/step - loss: 0.0667 - accuracy: 0.9722 - val_loss: 0.0190 - val_accuracy: 1.0000
Epoch 118/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0748 - accuracy: 0.9630 - val_loss: 0.0453 - val_accuracy: 1.0000
Epoch 119/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0633 - accuracy: 0.9722 - val_loss: 0.0157 - val_accuracy: 1.0000
Epoch 120/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0649 - accuracy: 0.9815 - val_loss: 0.0154 - val_accuracy: 1.0000
Epoch 121/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0682 - accuracy: 0.9722 - val_loss: 0.0131 - val_accuracy: 1.0000
Epoch 122/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0562 - accuracy: 0.9815 - val_loss: 0.0167 - val_accuracy: 1.0000
Epoch 123/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0579 - accuracy: 0.9907 - val_loss: 0.0127 - val_accuracy: 1.0000
Epoch 124/200
4/4 [======

4/4 [==============================] - 0s 10ms/step - loss: 0.0517 - accuracy: 0.9815 - val_loss: 0.0350 - val_accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0831 - accuracy: 0.9722 - val_loss: 0.0493 - val_accuracy: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0852 - accuracy: 0.9630 - val_loss: 0.0432 - val_accuracy: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0627 - accuracy: 0.9722 - val_loss: 0.0293 - val_accuracy: 1.0000
Epoch 179/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0632 - accuracy: 0.9722 - val_loss: 0.0207 - val_accuracy: 1.0000
Epoch 180/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0552 - accuracy: 0.9815 - val_loss: 0.0203 - val_accuracy: 1.0000
Epoch 181/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0544 - accuracy: 0.9815 - val_loss: 0.0136 - val_accuracy: 1.0000
Epoch 182/200
4/4 [======

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [27]:
from sklearn.neural_network import MLPClassifier

In [28]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [29]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver='adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습최대 횟수
            early_stopping=True, # 조기종료 활성화
            n_iter_no_change=10, # earlystopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start=False # True일 경우 이전학습에 이어서 학습
)

In [30]:
model.fit(train_X,train_y)

,hidden_layer_sizes,"(64, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,40
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,1000
,shuffle,True
,random_state,None


In [31]:
model.score(test_X, test_y)

0.8666666666666667

In [32]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [33]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [37]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X)
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,6,4
2,0,0,10


# 3. 클래스를 생성하여 모델함수 사용

In [45]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(units=50, activation=activation))
#         model.add(Dense(units=30, activation=activation))
#         model.add(Dense(units=3, activation='softmax'))
        # 모델구성
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=output_dim, activation='softmax'),
        ])
        # 학습설정
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [43]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1)

Epoch 1/50
4/4 [==============================] - 0s 36ms/step - loss: 1.7101 - accuracy: 0.3241 - val_loss: 1.2749 - val_accuracy: 0.4167
Epoch 2/50
4/4 [==============================] - 0s 8ms/step - loss: 1.2656 - accuracy: 0.3056 - val_loss: 1.2398 - val_accuracy: 0.2500
Epoch 3/50
4/4 [==============================] - 0s 6ms/step - loss: 1.1885 - accuracy: 0.3426 - val_loss: 1.1618 - val_accuracy: 0.2500
Epoch 4/50
4/4 [==============================] - 0s 11ms/step - loss: 1.0850 - accuracy: 0.5741 - val_loss: 1.0645 - val_accuracy: 0.5833
Epoch 5/50
4/4 [==============================] - 0s 10ms/step - loss: 1.0117 - accuracy: 0.6759 - val_loss: 0.9786 - val_accuracy: 0.5833
Epoch 6/50
4/4 [==============================] - 0s 5ms/step - loss: 0.9360 - accuracy: 0.6759 - val_loss: 0.8945 - val_accuracy: 0.5833
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 0.8608 - accuracy: 0.7130 - val_loss: 0.8374 - val_accuracy: 0.6667
Epoch 8/50
4/4 [===============

In [44]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100}%')

1/1 [==============================] - 0s 32ms/step - loss: 0.1459 - accuracy: 1.0000
정확도 : 100.0%


# 4. 함수형 API
- 병렬처리, Residual block

In [ ]:
# 기존 model 스타일1
model = Sequential()
model.add(Input(input_dim))
model.add(Dense(units=50, activation=activation))
model.add(Dense(units=30, activation=activation))
model.add(Dense(units=3, activation='softmax'))
# 기존 model 스타일2
model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=output_dim, activation='softmax'),
        ])

In [46]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_=Input(shape=(4,))
layer1=Dense(units=50, activation='relu')(input_)
layer2=Dense(units=30, activation='relu')(layer1)
output=Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_30 (Dense)            (None, 50)                250       
                                                                 
 dense_31 (Dense)            (None, 30)                1530      
                                                                 
 dense_32 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [50]:
# 병렬처리방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1,dense2,dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_33 (Dense)               (None, 50)           250         ['input_10[0][0]']               
                                                                                                  
 dense_34 (Dense)               (None, 80)           400         ['input_10[0][0]']               
                                                                                                  
 dense_35 (Dense)               (None, 30)           150         ['input_10[0][0]']               
                                                                                            

In [53]:
# 리지듀얼 블럭(Residual Block) : 딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1,dense2])
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_,outputs=output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_40 (Dense)               (None, 50)           250         ['input_12[0][0]']               
                                                                                                  
 dense_41 (Dense)               (None, 50)           2550        ['dense_40[0][0]']               
                                                                                                  
 add_1 (Add)                    (None, 50)           0           ['dense_40[0][0]',               
                                                                  'dense_41[0][0]']         

In [54]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_X, train_y, epochs=200, verbose=1)

Epoch 1/200
4/4 [==============================] - 0s 6ms/step - loss: 1.2998 - accuracy: 0.3333
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9863 - accuracy: 0.4667
Epoch 3/200
4/4 [==============================] - 0s 828us/step - loss: 0.9213 - accuracy: 0.6500
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 0.8931 - accuracy: 0.6667
Epoch 5/200
4/4 [==============================] - 0s 0s/step - loss: 0.8200 - accuracy: 0.6667
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 0.7592 - accuracy: 0.7083
Epoch 7/200
4/4 [==============================] - 0s 0s/step - loss: 0.7264 - accuracy: 0.6667
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 0.6907 - accuracy: 0.6667
Epoch 9/200
4/4 [==============================] - 0s 0s/step - loss: 0.6551 - accuracy: 0.6833
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.6333 - accuracy: 0.8917
Epoch 11/200
4/4 [============

4/4 [==============================] - 0s 2ms/step - loss: 0.1250 - accuracy: 0.9750
Epoch 85/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1228 - accuracy: 0.9750
Epoch 86/200
4/4 [==============================] - 0s 0s/step - loss: 0.1233 - accuracy: 0.9750
Epoch 87/200
4/4 [==============================] - 0s 5ms/step - loss: 0.1238 - accuracy: 0.9750
Epoch 88/200
4/4 [==============================] - 0s 0s/step - loss: 0.1211 - accuracy: 0.9667
Epoch 89/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1199 - accuracy: 0.9833
Epoch 90/200
4/4 [==============================] - 0s 0s/step - loss: 0.1171 - accuracy: 0.9750
Epoch 91/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1192 - accuracy: 0.9667
Epoch 92/200
4/4 [==============================] - 0s 0s/step - loss: 0.1156 - accuracy: 0.9667
Epoch 93/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1172 - accuracy: 0.9667
Epoch 94/200
4/4 [===================

4/4 [==============================] - 0s 5ms/step - loss: 0.0790 - accuracy: 0.9750
Epoch 168/200
4/4 [==============================] - 0s 0s/step - loss: 0.0801 - accuracy: 0.9750
Epoch 169/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0847 - accuracy: 0.9667
Epoch 170/200
4/4 [==============================] - 0s 0s/step - loss: 0.0800 - accuracy: 0.9750
Epoch 171/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0797 - accuracy: 0.9750
Epoch 172/200
4/4 [==============================] - 0s 0s/step - loss: 0.0788 - accuracy: 0.9833
Epoch 173/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0775 - accuracy: 0.9750
Epoch 174/200
4/4 [==============================] - 0s 0s/step - loss: 0.0808 - accuracy: 0.9667
Epoch 175/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0795 - accuracy: 0.9750
Epoch 176/200
4/4 [==============================] - 0s 0s/step - loss: 0.0792 - accuracy: 0.9833
Epoch 177/200
4/4 [==========

In [55]:
# 학습평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 1.0
